In [1]:
import os
import re
import pandas as pd
import collections

In [2]:
raw_data = os.path.join("..", "raw_data")
structure_data = os.path.join(".")

def structre_line(index, line):
    r = re.search(".*?instance: (.*?)\] (.*)",line)
    if r is None:
        return [None, None, None]
    entry = [index] + list(r.groups())
    entry[2] = re.sub("\d+.?\d* ?","",entry[2])
    entry[2] = re.sub(" /\S+","",entry[2])
    return entry
    
def structre_file(dir_in, dir_out, filename):
    filepath_in = os.path.join(dir_in, filename)
    lines = open(filepath_in).readlines()
    entries = [structre_line(index, line) for index, line in enumerate(lines)]
    return pd.DataFrame(entries, columns = ["line_index", "instance", "operation"])

def clean_df(df):
    df=df.dropna().reset_index(drop=True)
    def map_ops(df):
        df_op = df.groupby("operation").apply(lambda grp: pd.Series({"op_id_" : grp["line_index"].min()})).reset_index()
        df_op.sort_values("op_id_",inplace=True)
        df_op["op_id"] = ["s{}".format(i) for i in range(len(df_op))]
        df_op = df_op.drop("op_id_", axis=1)
        df = pd.merge(df, df_op, on="operation")
        return df
    df = map_ops(df)
    df = df.sort_values(by=["instance","line_index"])
    return df
        
for filename in os.listdir(raw_data):
    print(filename)
    df = structre_file(dir_in=raw_data, dir_out=structure_data, filename=filename)
    df = clean_df(df)
    df.to_csv(os.path.join(structure_data, "{}.csv".format(filename)))

openstack_val_n1
openstack_val_n2
